# Ungraded lab: Permutation Feature Importance
------------------------
 
Bienvenido, durante este laboratorio no calificado va a realizar Permutation Feature Importance en [wine dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine) usando scikit-learn. En particular:


1. 1. Entrenar un clasificador Random Forest en los datos.
2. 2. Calcular la puntuación de importancia de las características permutando cada característica.
3. 3. Vuelva a entrenar el modelo sólo con las características más importantes.
4. Compruebe otros clasificadores para comparar.

Empecemos.

## Inspect and pre-process the data

Empiece por actualizar scikit-learn a la última versión:

In [1]:
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 36.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


Now import the required dependencies and load the dataset:

In [1]:
import numpy as np
from sklearn.datasets import load_wine

# as_frame param requires scikit-learn >= 0.23
data = load_wine(as_frame=True)

# Print first rows of the data
data.frame.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


Este conjunto de datos se compone de 13 características numéricas y hay 3 clases diferentes de vino.

Ahora realice la división entrenamiento/prueba y normalice los datos utilizando [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html):

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Train / Test split
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, random_state=42)

# Instantiate StandardScaler
scaler = StandardScaler()

# Fit it to the train data
scaler.fit(X_train)

# Use it to transform the train and test data
X_train = scaler.transform(X_train)

# Notice that the scaler is trained on the train data to avoid data leakage from the test set
X_test = scaler.transform(X_test)

## Train the classifier

Ahora ajustará un [clasificador Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) con 10 estimadores y calculará la precisión media obtenida:

In [4]:
from sklearn.ensemble import RandomForestClassifier

# Fit the classifier
rf_clf = RandomForestClassifier(n_estimators=10, random_state=42).fit(X_train, y_train)

# Print the mean accuracy achieved by the classifier on the test set
rf_clf.score(X_test, y_test)

0.9111111111111111

Este modelo alcanzó una precisión media del 91%. Bastante bueno para un modelo sin ajuste fino.

# Permutation Feature Importance

Para llevar a cabo la técnica de inspección del modelo conocida como Permutation Feature Importance se utilizará la función integrada de scikit-learn [`permutation_importance`](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html#sklearn.inspection.permutation_importance).

Crearemos una función que, dado un clasificador, características y etiquetas, calcule la importancia de cada característica:

In [5]:
from sklearn.inspection import permutation_importance

def feature_importance(clf, X, y, top_limit=None):

  # Recuperar el objeto Bunch después de 50 repeticiones
  # n_repeats es el número de veces que se permutó cada característica para calcular la puntuación final
  bunch = permutation_importance(clf, X, y,
                                 n_repeats=50, random_state=42)

  #Importancia media de las características
  imp_means = bunch.importances_mean

  #Lista que contiene el índice de cada característica en orden descendente de importancia
  ordered_imp_means_args = np.argsort(imp_means)[::-1]

  # Si no hay límite imprimir todas las características
  if top_limit is None:
    top_limit = len(ordered_imp_means_args)

  # Imprimir información relevante
  for i, _ in zip(ordered_imp_means_args, range(top_limit)):
    name = data.feature_names[i]
    imp_score = imp_means[i]
    imp_std = bunch.importances_std[i]
    print(f"Feature {name} with index {i} has an average importance score of {imp_score:.3f} +/- {imp_std:.3f}\n")

La puntuación de importancia se calcula de forma que los valores más altos representan un mejor poder predictivo. Para saber exactamente cómo se calcula, consulta este [enlace](https://scikit-learn.org/stable/modules/permutation_importance.html#outline-of-the-permutation-importance-algorithm).

Ahora utilice la función `feature_importance` en el clasificador Random Forest y el conjunto de entrenamiento:

In [6]:
feature_importance(rf_clf, X_train, y_train)

Feature flavanoids with index 6 has an average importance score of 0.227 +/- 0.025

Feature proline with index 12 has an average importance score of 0.142 +/- 0.019

Feature color_intensity with index 9 has an average importance score of 0.112 +/- 0.023

Feature od280/od315_of_diluted_wines with index 11 has an average importance score of 0.007 +/- 0.005

Feature total_phenols with index 5 has an average importance score of 0.003 +/- 0.004

Feature malic_acid with index 1 has an average importance score of 0.002 +/- 0.004

Feature proanthocyanins with index 8 has an average importance score of 0.002 +/- 0.003

Feature hue with index 10 has an average importance score of 0.002 +/- 0.003

Feature nonflavanoid_phenols with index 7 has an average importance score of 0.000 +/- 0.000

Feature magnesium with index 4 has an average importance score of 0.000 +/- 0.000

Feature alcalinity_of_ash with index 3 has an average importance score of 0.000 +/- 0.000

Feature ash with index 2 has an aver

Parece que muchas de las características tienen una puntuación de importancia bastante baja. Esto indica que el poder predictivo de este conjunto de datos se concentra en unas pocas características.

Sin embargo, es importante tener en cuenta que este proceso se realizó para el conjunto de entrenamiento, por lo que la importancia de esta característica NO tiene en cuenta si la característica podría ayudar con el poder de generalización del modelo.

Para comprobarlo, repita el proceso para el conjunto de prueba:

In [7]:
feature_importance(rf_clf, X_test, y_test)

Feature flavanoids with index 6 has an average importance score of 0.202 +/- 0.047

Feature proline with index 12 has an average importance score of 0.143 +/- 0.042

Feature color_intensity with index 9 has an average importance score of 0.112 +/- 0.043

Feature alcohol with index 0 has an average importance score of 0.024 +/- 0.017

Feature magnesium with index 4 has an average importance score of 0.021 +/- 0.015

Feature od280/od315_of_diluted_wines with index 11 has an average importance score of 0.015 +/- 0.018

Feature hue with index 10 has an average importance score of 0.013 +/- 0.018

Feature total_phenols with index 5 has an average importance score of 0.002 +/- 0.016

Feature nonflavanoid_phenols with index 7 has an average importance score of 0.000 +/- 0.000

Feature alcalinity_of_ash with index 3 has an average importance score of 0.000 +/- 0.000

Feature malic_acid with index 1 has an average importance score of -0.002 +/- 0.017

Feature ash with index 2 has an average imp

Observe que las características más importantes son las mismas en ambos conjuntos. Sin embargo, características como **alcohol**, que no se consideró importante para el conjunto de entrenamiento, es mucho más importante cuando se utiliza el conjunto de pruebas. Esto indica que esta característica contribuirá al poder de generalización del modelo.

**Si una característica se considera importante en el conjunto de entrenamiento, pero no en el de pruebas, es probable que el modelo se ajuste en exceso.

## Re-train the model with the most important features

Ahora volverá a entrenar el clasificador Random Forest sólo con las 3 características más importantes. 

En este caso son las mismas para ambos conjuntos:

In [8]:
print("On TRAIN split:\n")
feature_importance(rf_clf, X_train, y_train, top_limit=3)

print("\nOn TEST split:\n")
feature_importance(rf_clf, X_test, y_test, top_limit=3)

On TRAIN split:

Feature flavanoids with index 6 has an average importance score of 0.227 +/- 0.025

Feature proline with index 12 has an average importance score of 0.142 +/- 0.019

Feature color_intensity with index 9 has an average importance score of 0.112 +/- 0.023


On TEST split:

Feature flavanoids with index 6 has an average importance score of 0.202 +/- 0.047

Feature proline with index 12 has an average importance score of 0.143 +/- 0.042

Feature color_intensity with index 9 has an average importance score of 0.112 +/- 0.043



In [9]:
# Preserve only the top 3 features
X_train_top_features = X_train[:,[6, 9, 12]]
X_test_top_features = X_test[:,[6, 9, 12]]

# Re-train with only these features
rf_clf_top = RandomForestClassifier(n_estimators=10, random_state=42).fit(X_train_top_features, y_train)

# Compute mean accuracy achieved
rf_clf_top.score(X_test_top_features, y_test)

0.9333333333333333

Observe que al utilizar sólo las 3 características más importantes, el modelo alcanzó una precisión media incluso superior a la del modelo que utilizaba las 13 características. 


Recuerde que la característica **alcohol** no se consideró importante en la división de entrenamiento, pero usted tenía la hipótesis de que contenía información importante para la generalización del modelo. 

Añada esta característica y vea cómo funciona el modelo:

In [10]:
# Preserve only the top 3 features
X_train_top_features = X_train[:,[0, 6, 9, 12]]
X_test_top_features = X_test[:,[0, 6, 9, 12]]

# Re-train with only these features
rf_clf_top = RandomForestClassifier(n_estimators=10, random_state=42).fit(X_train_top_features, y_train)

# Compute mean accuracy achieved
rf_clf_top.score(X_test_top_features, y_test)

1.0

¡Wow! Al añadir esta función adicional, se obtiene una precisión media del 100%. ¡Notable! Parece que esta característica proporcionó información importante que ayudó al modelo a generalizar mejor.

## Try out other classifiers

El proceso de permutación de la importancia de las características también depende del clasificador que se utilice. Dado que los distintos clasificadores siguen reglas de clasificación diferentes, es natural suponer que considerarán importantes o insignificantes características diferentes.

Para comprobarlo, pruebe otros clasificadores:

In [11]:
from sklearn.svm import SVC
from sklearn.linear_model import Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier

# Select 4 new classifiers
clfs = {"Laso": Lasso(alpha=0.05), 
        "Ridge": Ridge(), 
        "Decision Tree": DecisionTreeClassifier(), 
        "Support Vector": SVC()}


# Compute feature importance on the test set given a classifier
def fit_compute_importance(clf):
  clf.fit(X_train, y_train)
  print(f"📏 Mean accuracy score on the test set: {clf.score(X_test, y_test)*100:.2f}%\n")
  print("🔝 Top 4 features when using the test set:\n")
  feature_importance(clf, X_test, y_test, top_limit=4)


# Print results
for name, clf in clfs.items():
  print("====="*20)
  print(f"➡️ {name} classifier\n")
  fit_compute_importance(clf)

➡️ Laso classifier

📏 Mean accuracy score on the test set: 86.80%

🔝 Top 4 features when using the test set:

Feature flavanoids with index 6 has an average importance score of 0.323 +/- 0.055

Feature proline with index 12 has an average importance score of 0.203 +/- 0.035

Feature od280/od315_of_diluted_wines with index 11 has an average importance score of 0.146 +/- 0.030

Feature alcalinity_of_ash with index 3 has an average importance score of 0.038 +/- 0.014

➡️ Ridge classifier

📏 Mean accuracy score on the test set: 88.71%

🔝 Top 4 features when using the test set:

Feature flavanoids with index 6 has an average importance score of 0.445 +/- 0.071

Feature proline with index 12 has an average importance score of 0.210 +/- 0.035

Feature color_intensity with index 9 has an average importance score of 0.119 +/- 0.029

Feature od280/od315_of_diluted_wines with index 11 has an average importance score of 0.111 +/- 0.026

➡️ Decision Tree classifier

📏 Mean accuracy score on the tes

Parece que los **flavanoides** y la **prolina** son muy importantes en todos los clasificadores. Sin embargo, existe variabilidad de un clasificador a otro en cuanto a las características que se consideran más importantes.

-----------------------------
**Ahora debería tener una comprensión más clara de lo que es la Importancia de las Características de Permutación, por qué es útil y cómo implementar esta técnica utilizando Scikit-learn. 

**Sigan así.**